In [1]:
DATA_DIR = '/home/nsuprotivniy/Documents/Работа/OKru/Antispam/data/'
SAMPLE_SIZE = -1 # -1 если использовать все даннные

In [2]:
!pip install pyarrow
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import seaborn as sns
import user_agents
%matplotlib inline
from matplotlib import pyplot as plt
pd.set_option("display.precision", 2)

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import export_graphviz

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
http = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180217_1718_parsedUA.parquet').to_pandas()

In [4]:
http.dtypes

browserId                   float64
from                         object
ip                           object
method                       object
operation                    object
referrer                     object
requestType                  object
timestamp                     int64
to                           object
url                          object
userAgent                    object
userId                       object
hour                          int32
isBot                         int64
userAgentBrowserFamily       object
userAgentBrowserVersion     float64
userAgentDeviceBrand         object
userAgentDeviceFamily        object
userAgentDeviceModel         object
userAgentIsBot                 bool
userAgentIsMobile              bool
userAgentIsPC                  bool
userAgentIsTablet              bool
userAgentIsTouchCapable        bool
userAgentOSFamily            object
userAgentOSVersion          float64
userAgentBrowserVersion0     object
userAgentBrowserVersion1    

In [12]:
http.userAgentOSVersion0 = http.userAgentOSVersion0.fillna('0').astype('str').fillna('0)

SyntaxError: EOL while scanning string literal (<ipython-input-12-9077281f1ef7>, line 1)

In [10]:
http

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,...,userAgentIsTablet,userAgentIsTouchCapable,userAgentOSFamily,userAgentOSVersion,userAgentBrowserVersion0,userAgentBrowserVersion1,userAgentBrowserVersion2,userAgentOSVersion0,userAgentOSVersion1,userAgentOSVersion2
0,8.29e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,...,False,False,Windows 8.1,8.29e+18,64.0,0.0,3282,nan,nan,nan
0,8.29e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,...,False,False,Windows 10,8.29e+18,16.0,16299.0,nan,nan,nan,nan
0,-8.65e+18,None,nUiMrZ2qqXpL+XEgVSVfjQ==,POST,videoStatNew,https://st.mycdn.me/static/MegaPlayer/10-8-31/...,REQ,1518791037258,None,https://ok.ru/dk,...,False,False,Windows 8.1,-8.65e+18,64.0,0.0,3282,nan,nan,nan
0,-8.65e+18,None,nUiMrZ2qqXpL+XEgVSVfjQ==,POST,videoStatNew,https://st.mycdn.me/static/MegaPlayer/10-8-31/...,REQ,1518791037258,None,https://ok.ru/dk,...,False,False,Windows 10,-8.65e+18,16.0,16299.0,nan,nan,nan,nan
1,8.29e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,...,False,False,Windows 8.1,8.29e+18,64.0,0.0,3282,nan,nan,nan
1,8.29e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,...,False,False,Windows 7,8.29e+18,17.0,11.0,1,nan,nan,nan
1,5.55e+18,None,lc5Amfakdl9L2+kl6Mh/8A==,POST,videoStatNew,https://ok.ru/,REQ,1518793457723,None,https://ok.ru/dk,...,False,False,Windows 8.1,5.55e+18,64.0,0.0,3282,nan,nan,nan
1,5.55e+18,None,lc5Amfakdl9L2+kl6Mh/8A==,POST,videoStatNew,https://ok.ru/,REQ,1518793457723,None,https://ok.ru/dk,...,False,False,Windows 7,5.55e+18,17.0,11.0,1,nan,nan,nan
2,7.90e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,...,False,False,Windows 10,7.90e+17,47.0,0.0,2529,nan,nan,nan
2,7.90e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,...,False,False,Windows 7,7.90e+17,18.0,1.0,1,nan,nan,nan
